In [1]:
import pickle
import numpy as np
import matplotlib.pyplot as plt
import cv2
from shapely.geometry import Polygon
import networkx as nx
import torch
import torch.nn.functional as F
from skimage import transform
import matplotlib.image as mpimg
from torch.utils.data import DataLoader
from floortrans.models import get_model
from floortrans.loaders import FloorplanSVG, DictToTensor, Compose, RotateNTurns
from floortrans.plotting import segmentation_plot, polygons_to_image, draw_junction_from_dict, discrete_cmap
#discrete_cmap()
from floortrans.post_prosessing import split_prediction, get_polygons, split_validation
from mpl_toolkits.axes_grid1 import AxesGrid

In [2]:
rot = RotateNTurns() #
room_classes = ["Background", "Outdoor", "Wall", "Kitchen", "Living Room" ,"Bed Room", "Bath", "Entry", "Railing", "Storage", "Garage", "Undefined"]
icon_classes = ["No Icon", "Window", "Door", "Closet", "Electrical Applience" ,"Toilet", "Sink", "Sauna Bench", "Fire Place", "Bathtub", "Chimney"]
room_classes.append("Door")
data_folder = 'data/cubicasa5k/'
data_file = 'test.txt'
normal_set = FloorplanSVG(data_folder, data_file, format='txt', original_size=True)
data_loader = DataLoader(normal_set, batch_size=1, num_workers=0)
data_iter = iter(data_loader)
# Setup Model
model = get_model('hg_furukawa_original', 51)
n_classes = 44
split = [21, 12, 11]

In [3]:
def isolate_class(rooms, CLASS: int):
    template = np.zeros_like(rooms)
    rows, cols = np.where(rooms == CLASS)
    template[rows, cols] = 1
    return template
    
bad=[0, 1, 2, 8, 11]
good=[3,4,5,6,7,9,10,12]